In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "./data/events-stream"
val modelPath = "./models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = ./data/events-stream
modelPath = ./models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
val myModel = PipelineModel.load("./models/spark-ml-model")

inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>

    val predictionsDF = myModel.transform(batchDF)
    val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
    predictionsDF.select($"tweet", getProbability($"probability").alias("clean_probability")).show
    
}.start()

myModel = pipeline_e1c3d5ad6049


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@37c4e9f8

+--------------------+-------------------+
|               tweet|  clean_probability|
+--------------------+-------------------+
|Doing nothing tod...|0.44577385267091846|
|Pissed that twitp...|0.44577385267091846|
|my wifi isnt work...| 0.5041568910120379|
|come on... drop b...| 0.5177776891306454|
|@iLeoCastro  I'm ...| 0.5041568910120379|
|The CD drive isn'...|0.44577385267091846|
|@suziam not sure ...|0.46957254331679155|
|did a jeaga bomb ...| 0.5041568910120379|
|Please subscribe ...| 0.5870866341203763|
|is discussing bll...| 0.5041568910120379|
|#3turnoffwords gr...| 0.5177776891306454|
|         Follow ME! | 0.5041568910120379|
|Watched MTV award...| 0.5713927240888151|
|@mileycyrus http:...| 0.5428149328654575|
|    uqhh ii want DD |0.44577385267091846|
|@brennybeee aw im...|  0.451571725910382|
|I want a orange s...| 0.5426401961604143|
|@mrscabeen One of...| 0.5041568910120379|
+--------------------+-------------------+

+--------------------+------------------+
|          